## Import package and Setting

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
import pathlib
import tensorflow_addons as tfa
import efficientnet.tfkeras
from IPython.core.display import display, HTML 

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "14,15"
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_memory_growth(gpus[1], True)
display(HTML("<style>.container { width:98% !important; }</style>")) 

## Load image data

In [3]:
data_path = "flower_photos/"

In [4]:
data_root = pathlib.Path(data_path)

In [5]:
label_names = sorted(item.name for item in data_root.glob("*/") if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("label names:", label_names)
print("label to index:", label_to_index)

label names: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
label to index: {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [6]:
all_image_paths = list(data_root.glob("*/*"))
all_image_paths = [str(path) for path in all_image_paths]

In [7]:
random.shuffle(all_image_paths)

In [8]:
train_image_paths = all_image_paths[500:]
valid_image_paths = all_image_paths[:500]

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [10]:
def preprocessing_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, tf.random.uniform([2], minval=224, maxval=256, dtype=tf.int32))
    image = tf.image.random_crop(image ,[224, 224, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tfa.image.rotate(image, tf.random.uniform([], -30, 30))
    
    image -= tf.reduce_mean(image)
    image /= (tf.math.reduce_std(image)+1e-6)
    return image

In [11]:
def load_and_preprocessing_image(path):
    image = tf.io.read_file(path)
    return preprocessing_image(image)

In [12]:
def dataset_generate(image_paths, batch_size, label_to_index, AUTOTUNE):
    image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in image_paths]
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    image_ds = path_ds.map(load_and_preprocessing_image, num_parallel_calls=AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(image_labels, tf.float32))
    image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
    image_counts = len(image_paths)
    print("Step:", int(image_counts/batch_size))
    image_label_ds = image_label_ds.shuffle(buffer_size=image_counts).repeat().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
    return image_label_ds

In [13]:
train_datasets = dataset_generate(train_image_paths, 16, label_to_index, AUTOTUNE)
valid_datasets = dataset_generate(valid_image_paths, 16, label_to_index, AUTOTUNE)

Step: 198
Step: 31


## Build Model

In [14]:
strategy = tf.distribute.MirroredStrategy()

In [15]:
with strategy.scope():
    model = tf.keras.Sequential()
    conv_base = tf.keras.models.load_model("/raid/peterchen/Python_Tool/efficientnet_weight/no_top/EfficientNetB0.h5")
    model.add(conv_base)
    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Dense(5, activation="softmax"))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=["acc"])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, None, None, 1280)  4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 4,055,969
Trainable params: 4,013,953
Non-trainable params: 42,016
_________________________________________________________________


## Training

In [17]:
model.fit(train_datasets, steps_per_epoch=1200, 
          validation_data=valid_datasets, validation_steps=50,
          epochs=30, verbose=1, shuffle=True)

Train for 1200 steps, validate for 50 steps
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
1200/1200 [==============================] - 123s 102ms/step - loss: 0.4024 - acc: 0.8549 - val_loss: 0.3379 - val_acc: 0.8938
Epoch 2/30
1200/1200 [==============================] - 82s 68ms/step - loss: 0.2317 - acc: 0.9171 - val_loss: 0.2869 - val_acc: 0.9112
Epoch 3/30
1200/1200 [==============================] - 82s 68ms/step - loss: 0.1847 - acc: 0.9339 - val_loss: 0.3602 - val_acc: 0.8775
Epoch 4/30
1200/1200 [==============================] - 82s 68ms/step - loss: 0.1497 - acc: 0.9454 - val_loss: 0.4625 - val_acc: 0.8863
Epoch 5/30
1200/1200 [==============================] - 82s 68ms/step - loss: 0.1324 - acc: 0.9540 - val

## Save model

In [18]:
model.save("flower_efficientnet.h5", include_optimizer=False)